In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)    

In [3]:
x = []
i = 0
for i in range(5):
    path = "data_batch_" + str(i + 1)
    with open(path, 'rb') as f:
        batch = pickle.load(f,encoding='latin1')
    x.append(batch['data'])  
x = np.concatenate(x)/255.0
print(x.shape)



(50000, 3072)


In [4]:
X = tf.placeholder(tf.float32, shape = [None, 3072])

In [5]:
D_W1 = tf.Variable(xavier_init([3072, 256]))
D_B1 = tf.Variable(tf.zeros(shape = [256]))
D_W2 = tf.Variable(xavier_init([256, 128]))
D_B2 = tf.Variable(tf.zeros(shape = [128]))
D_W3 = tf.Variable(xavier_init([128,1]))
D_B3 = tf.Variable(tf.zeros(shape = [1]))

theta_D = [D_W1, D_B1, D_W2, D_B2, D_W3, D_B3]

In [6]:
Z = tf.placeholder(tf.float32, shape = [None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_B1 = tf.Variable(tf.zeros(shape = [128]))
G_W2 = tf.Variable(xavier_init([128, 256]))
G_B2 = tf.Variable(tf.zeros(shape = [256]))
G_W3 = tf.Variable(xavier_init([256, 3072]))
G_B3 = tf.Variable(tf.zeros(shape = [3072]))

theta_G = [G_W1, G_B1, G_W2, G_B2, G_W3, G_B3]

In [7]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size = [m, n])

In [8]:
def discriminator(p):
    D_h1 = tf.nn.relu(tf.matmul(p, D_W1) + D_B1)
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_B2)
    D_logit = tf.matmul(D_h2, D_W3) + D_B3

    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit


In [9]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_B1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_B2)
    G_log_prob = tf.matmul(G_h2, G_W3) + G_B3

    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [10]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        R = sample[0:1024].reshape(32,32)
        G = sample[1024:2048].reshape(32,32)
        B = sample[2048:].reshape(32,32)

        img = np.dstack((R,G,B))   
        plt.imshow(img, cmap='hsv', interpolation='nearest')
    return fig

In [11]:
G_sample = generator(Z)
D_real, D_real_logit = discriminator(X)
D_fake, D_fake_logit = discriminator(G_sample)

In [12]:
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_real_logit, labels = tf.ones_like(D_real_logit)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logit, labels = tf.zeros_like(D_fake_logit)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logit, labels = tf.ones_like(D_fake_logit)))

In [13]:
D_solver = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(D_loss, var_list = theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(G_loss, var_list = theta_G)

In [15]:
mb_size = 64
Z_dim = 100
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('outc/'):
    os.makedirs('outc/')

In [ ]:
i = 0
for j in range(100000):
    print(j)
    if j % 50 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})        
        fig = plot(samples)
        plt.savefig('outc/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)    
    k = 0
    while k < 50000:
        start = k
        end = k + mb_size
        X_mb = x[start:end,:]                
        k += mb_size
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict = {X: X_mb, Z: sample_Z(mb_size, Z_dim)})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict = {Z: sample_Z(mb_size, Z_dim)})
        

            
    if j % 50 == 0 :
        print('Iteration: {}'.format(j))
        print('D_loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()
    


0
Iteration: 0
D_loss: 0.2014
G_loss: 2.907

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Iteration: 50
D_loss: 0.9942
G_loss: 2.919

51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Iteration: 100
D_loss: 0.6074
G_loss: 2.76

101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
Iteration: 150
D_loss: 0.8045
G_loss: 2.092

151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Iteration: 200
D_loss: 2.225
G_loss: 1.971

201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
22

In [ ]:
saver=tf.train.Saver()
save_path = saver.save(sess,"./model1.ckpt")